# Model Benchmarking RMSE Summary Table (all queries/metrics)

This notebook generates summary RMSEs for all QueryName and Metric combinations in SimulatedQueryMetrics.csv, for ARIMA, Prophet, LSTM, Random Forest, and XGBoost.

In [ ]:
# !pip install numpy pandas scikit-learn xgboost statsmodels prophet tensorflow

In [2]:
import numpy as np
import pandas as pd
import time
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [4]:
csv_file = 'SimulatedQueryMetrics.csv'
df = pd.read_csv(csv_file)
df['MetricDate'] = pd.to_datetime(df['MetricDate'])
metrics = ['CPU', 'LatencyMs', 'LogicalReads']
queries = df['QueryName'].unique()
lags = 7

def create_lag_features(df, lags=7, val_col='y'):
    df = df.copy()
    for lag in range(1, lags+1):
        df[f'lag_{lag}'] = df[val_col].shift(lag)
    df = df.dropna().reset_index(drop=True)
    return df

## Run Benchmarking for All Query-Metric Combinations

In [6]:
summary_rows = []
for query in queries:
    for metric in metrics:
        dfq = df[df['QueryName'] == query].copy().sort_values('MetricDate').reset_index(drop=True)
        if dfq[metric].isnull().all() or len(dfq) < lags + 10:
            continue  # skip if not enough data
        # Data prep
        dfq_sub = dfq[['MetricDate', metric, 'QueryVariant']].rename(columns={'MetricDate':'ds', metric:'y'})
        dfq_lagged = create_lag_features(dfq_sub, lags=lags, val_col='y')
        split = int(len(dfq_lagged) * 0.8)
        train_df = dfq_lagged.iloc[:split]
        test_df = dfq_lagged.iloc[split:]
        X_train = train_df[[f'lag_{i}' for i in range(1, lags+1)]]
        y_train = train_df['y']
        X_test = test_df[[f'lag_{i}' for i in range(1, lags+1)]]
        y_test = test_df['y']

        # ARIMA
        try:
            arima_train = train_df['y']
            arima_test = test_df['y']
            arima_model = ARIMA(arima_train, order=(lags, 0, 0))
            arima_fit = arima_model.fit()
            arima_pred = arima_fit.forecast(steps=len(arima_test))
            arima_rmse = mean_squared_error(arima_test, arima_pred, squared=False)
        except Exception as e:
            arima_rmse = np.nan

        # Prophet
        try:
            prophet_train = train_df[['ds', 'y']]
            prophet_test = test_df[['ds', 'y']]
            m = Prophet()
            m.fit(prophet_train)
            future = prophet_test[['ds']]
            forecast = m.predict(future)
            prophet_rmse = mean_squared_error(prophet_test['y'], forecast['yhat'], squared=False)
        except Exception as e:
            prophet_rmse = np.nan

        # LSTM
        try:
            X_train_lstm = X_train.values.reshape((-1, lags, 1))
            X_test_lstm = X_test.values.reshape((-1, lags, 1))
            model = Sequential()
            model.add(LSTM(32, input_shape=(lags, 1)))
            model.add(Dense(1))
            model.compile(optimizer='adam', loss='mse')
            model.fit(X_train_lstm, y_train, epochs=10, batch_size=32, verbose=0)
            lstm_pred = model.predict(X_test_lstm).flatten()
            lstm_rmse = mean_squared_error(y_test, lstm_pred, squared=False)
        except Exception as e:
            lstm_rmse = np.nan

        # Random Forest
        try:
            rf = RandomForestRegressor(n_estimators=100, random_state=42)
            rf.fit(X_train, y_train)
            rf_pred = rf.predict(X_test)
            rf_rmse = mean_squared_error(y_test, rf_pred, squared=False)
        except Exception as e:
            rf_rmse = np.nan

        # XGBoost
        try:
            xgb = XGBRegressor(n_estimators=100, random_state=42, verbosity=0)
            xgb.fit(X_train, y_train)
            xgb_pred = xgb.predict(X_test)
            xgb_rmse = mean_squared_error(y_test, xgb_pred, squared=False)
        except Exception as e:
            xgb_rmse = np.nan

        summary_rows.append({
            'Query': query,
            'Metric': metric,
            'ARIMA': np.round(arima_rmse, 2),
            'Prophet': np.round(prophet_rmse, 2),
            'LSTM': np.round(lstm_rmse, 2),
            'Random Forest': np.round(rf_rmse, 2),
            'XGBoost': np.round(xgb_rmse, 2)
        })

05:16:46 - cmdstanpy - INFO - Chain [1] start processing
05:16:46 - cmdstanpy - INFO - Chain [1] done processing


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step


05:16:58 - cmdstanpy - INFO - Chain [1] start processing
05:16:58 - cmdstanpy - INFO - Chain [1] done processing


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step


05:17:09 - cmdstanpy - INFO - Chain [1] start processing
05:17:09 - cmdstanpy - INFO - Chain [1] done processing


...

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step


05:17:33 - cmdstanpy - INFO - Chain [1] start processing
05:17:34 - cmdstanpy - INFO - Chain [1] done processing


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step


05:17:45 - cmdstanpy - INFO - Chain [1] start processing
05:17:45 - cmdstanpy - INFO - Chain [1] done processing


10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step


## Display Summary Table

In [8]:
summary_df = pd.DataFrame(summary_rows)
# For display: arrange columns like the sample
summary_df = summary_df[['Query', 'Metric', 'ARIMA', 'Prophet', 'LSTM', 'Random Forest', 'XGBoost']]
display(summary_df)
summary_df.to_csv('model_benchmark_summary.csv', index=False)

,Query,Metric,ARIMA,Prophet,LSTM,Random Forest,XGBoost
0,Q1,CPU,12.07,13.30,37.72,8.38,9.40
1,Q1,LatencyMs,23.31,37.78,181.08,12.48,14.98
2,Q1,LogicalReads,20.70,19.28,153.17,14.76,16.55
3,Q2,CPU,11.99,10.89,43.59,8.47,9.09
4,Q2,LatencyMs,25.22,35.18,187.25,12.84,14.89
5,Q2,LogicalReads,21.46,19.96,158.12,15.48,16.87
